# Загрузка данных
будет достаточно лишь одной таблицы с рейтингами, поэтому загрузим скачанный датасет из csv

In [177]:
import pandas as pd
df = pd.read_csv("C:\datasets\ml-latest\\ratings.csv")

# Разбиение данных
Возьмем стандартные 10% данных на валидацию и 10% на тест, но разделение будем производить по времени(timestamp), чтобы понимать как модель будет себя вести на данных "из будущего"
При этом валидационные данные будут использоваться в процессе обучения для оценки функции потерь после каждой эпохи


In [178]:
filtered_data = (
    df.filter(["timestamp", "userId", "movieId", "rating"])
    .sort_values("timestamp")
    .astype({"userId": int, "movieId": int, "rating": float})
    .drop(columns=["timestamp"])
)


In [179]:
test_range = int(len(filtered_data)*0.9)
train = filtered_data.iloc[:test_range]
test = filtered_data.iloc[test_range:]

In [180]:
X_train = train.drop(columns=["rating"])
y_train = train["rating"]
X_test = test.drop(columns=["rating"])
y_test = test["rating"]

In [181]:
import numpy
all_users = train["userId"].unique()
all_movies = train["movieId"].unique()

print("train users vocab len", len(all_users))
print("train movies vocab len", len(all_movies))


train users vocab len 263764
train movies vocab len 41330


# Архитектура
Для составления модели на основе эмбеддингов нам будет достаточно трех признаков userId, movieId, rating
на входе два катериальных признака (user_id, movie_id), затем мы проводим эмбеддинг каждого из них в размерность равную корень 4 степени из длины словаря (что является одной из возможных эмпирик), затем с помощью matrix factorization вычисляем user_rating

После обучения данного рекомендателя мы получаем обученные эмбеддинги для юзеров и фильмов, при этом эмбеддинги юзера и эмбеддинги подходящих ему фильмов будут близки. На их основе можно построить K-Nearest классификатор, который по эмбеддингу юзера определит n подходящих ему фильмов
## Детали реализации
Нормируем вывод модели в диапазоне от 1 до 5, также добавим bias для каждого юзера (для регулировки склонности ставить только низкие/высокие/одинаковые оценки) и bias для каждого фильма (склонность получать только плохие/хорошие оценки)

In [196]:
import tensorflow as tf
emb_dim = int(len(all_users) ** (1/4))
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=emb_dim)(user_as_integer)
user_vec = tf.keras.layers.Flatten(name='FlattenUser')(user_embedding)
user_model = tf.keras.Model(inputs=user_input, outputs=user_vec)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1, name="user_bias")(user_as_integer)

movie_input = tf.keras.layers.Input(shape=(1,), name="movie")
movie_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_movies)(movie_input)
movie_embedding = tf.keras.layers.Embedding(input_dim=len(all_movies) + 1, output_dim=emb_dim)(movie_as_integer)
movie_vec = tf.keras.layers.Flatten(name='FlattenMovie')(movie_embedding)
movie_model = tf.keras.Model(inputs=movie_input, outputs=movie_vec)
movie_bias = tf.keras.layers.Embedding(input_dim=len(all_movies) + 1, output_dim=1, name="movie_bias")(movie_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, movie_embedding])
add = tf.keras.layers.Add(name="sum")([dot, user_bias, movie_bias])
flatten = tf.keras.layers.Flatten(name="flatten")(add)
squash = tf.keras.layers.Lambda(lambda x: 4 * tf.nn.sigmoid(x) + 1, name="squash")(flatten)

model = tf.keras.Model(inputs=[user_input, movie_input], outputs=squash)

model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [183]:
model.fit(
    x={"user": X_train["userId"], "movie": X_train["movieId"]},
    y=y_train.values,
    batch_size=16000,
    epochs=20,
    validation_split=0.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(min_delta=0.005,patience=1, restore_best_weights=True)],
)

Epoch 1/20
1406/1406 [==============================] - 57s 39ms/step - loss: 1.0602 - mean_absolute_error: 0.8289 - val_loss: 1.1066 - val_mean_absolute_error: 0.8225
Epoch 2/20
1406/1406 [==============================] - 55s 39ms/step - loss: 0.8947 - mean_absolute_error: 0.7408 - val_loss: 1.0827 - val_mean_absolute_error: 0.8041
Epoch 3/20
1406/1406 [==============================] - 59s 42ms/step - loss: 0.8650 - mean_absolute_error: 0.7245 - val_loss: 1.0730 - val_mean_absolute_error: 0.7973
Epoch 4/20
1406/1406 [==============================] - 59s 42ms/step - loss: 0.8473 - mean_absolute_error: 0.7153 - val_loss: 1.0673 - val_mean_absolute_error: 0.7934
Epoch 5/20
1406/1406 [==============================] - 59s 42ms/step - loss: 0.8348 - mean_absolute_error: 0.7089 - val_loss: 1.0634 - val_mean_absolute_error: 0.7907


# Метрики
Для нас первостепенное значение имеет метрика r², которая показывает зависимость предсказзной оценки от входных данных пользоватея и фильма.
Также можно обратить внимание на среднюю абсолютную ошибку, которая означает, что мы отклоняемся от истинного рейтинга в среднем примерно на n звезды.

In [186]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

r2 = r2_score(
    y_test,
    model.predict({"user": X_test["userId"], "movie": X_test["movieId"]}, batch_size=16000).ravel(),
)

print(r2)

mae = mean_absolute_error(
    y_test,
    model.predict({"user": X_test["userId"], "movie": X_test["movieId"]}, batch_size=16000).ravel(),
)
print(mae)


174/174 [==============================] - 1s 3ms/step
0.08042698004501514
1562/1562 [==============================] - 5s 3ms/step
0.23851592364181573
174/174 [==============================] - 1s 3ms/step
0.8069650510406239


# Результаты
r² равен 8%, что в 3 раза лучше чем результат Градиентного Бустинга без применения эмбеддингов
Средняя абсолютная ошибка составляет ~0.8.

Стоит отметить, что r² для поддатасета из первого миллиона записей составляет 24%.

Скорее всего, невысокий r² для полного датасета на тесте объясняется cold problem, т.к тренировочные данные содержат лишь 85% всех юзеров и 58% всех фильмов

In [187]:
test_range = int(len(filtered_data)*0.8)

train = filtered_data.iloc[:test_range]
test = filtered_data.iloc[test_range:]

train_unique_users_len = len(train["userId"].unique())
train_unqie_movie_len = len(train["movieId"].unique())

all_unqie_users_len = len(filtered_data["userId"].unique())
all_unqie_films_len = len(filtered_data["movieId"].unique())


print("доля юзеров в train", train_unique_users_len/all_unqie_users_len)
print("доля фильмов в train:", train_unqie_movie_len/all_unqie_films_len)

доля юзеров в train 0.8547671840354767
доля фильмов в train: 0.5872070366865223


# Достаем ембеддинги фильмов

In [194]:
all_movie_emb = movie_model.predict(all_movies)

1292/1292 [==============================] - 1s 953us/step


# Обучаем K-соседей пишем функцию для предсказания топ-n фильмов

In [189]:
knn_train_label = all_movies

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(all_movie_emb, knn_train_label)

In [191]:
def recommend_movies(user_id, n):
    user_embedding = user_model.predict([user_id])
    _, indices = clf.kneighbors(user_embedding,  n_neighbors=n)
    return indices

In [192]:
TEST_USER_ID = 105

In [193]:
print(recommend_movies(TEST_USER_ID, 5))

1/1 [==============================] - 0s 58ms/step
[[28754 35957 15295 33624 19996]]


# Выводы
Одно из главных преимуществ в том, что мы можем применять эту модель в большинстве контекстов, поскльку нам необходимы только данные о взаимодействии (рейтинг) юзеров и фильмов. Нам больше не нужно знать ничего о пользователях и фильмах, таких как возраст, пол, жанр и т. д., поэтому обычно мы можем приступить к работе немедленно.

Цена, которую мы платим за это, заключается в том, что мы не можем выводить осмысленные эмбеддинги для неизвестных пользователей или фильмов — cold start problem. Модель что-то выдаст, но качество будет ужасное.